In [ ]:
from stereo3d import stereo3d_read_from_zips as std_read
from parsivel import pars_read_from_zips as pars_read
from aux_funcs.extract_events import is_event
from multifractal_analysis.general import split_field
from multifractal_analysis.spectral_analysis import get_beta_and_h
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from pathlib import Path
import numpy as np 
stereo_datafolder = "/home/marcio/stage_project/data/Daily_raw_data_3D_stereo/"
pars_datafolder = "/home/marcio/stage_project/data/Pars_1/"
beg = 20230821000000
end = 20231022235959

# Read all the data and extract one event

In [ ]:
# Read the main parsivel series
parsivel_series = pars_read(beg, end, pars_datafolder)
stereo_series = std_read(beg, end, stereo_datafolder)

In [ ]:
plt.plot(parsivel_series.rain_rate)

In [ ]:
figure = plt.figure()
new_fields = split_field(parsivel_series.rain_rate,8, 120)
n_samples = new_fields.shape[1]
n = int(np.ceil(np.sqrt(n_samples)))
figure.set_size_inches((4*n, 4*n))
for i in range(n_samples):
    ax = figure.add_subplot(n,n,i+1)
    ax.plot(new_fields[:,i])

In [ ]:
beta, h = get_beta_and_h(new_fields)
beta, h

In [ ]:
pars_events_series = parsivel_series.find_events(15,0.7)
str_events_series = is_event(stereo_series.rain_rate(30)[:-1], 15, 0.7)
plt.plot(pars_events_series)
plt.plot(str_events_series/2)

In [ ]:
parsivel_events = parsivel_series.colect_events(np.logical_and(pars_events_series,str_events_series))
len(parsivel_events)

In [ ]:
n = int(np.ceil(len(parsivel_events)**0.5))
figure = plt.figure()
figure.set_size_inches((n*3 +3, n*3 + 3))
figure.set_constrained_layout(True)
for i, event in enumerate(parsivel_events):
    ax = figure.add_subplot(n,n, i + 1)
    ax.plot(event.rain_rate)

#### Select event from Stereo 3D

In [ ]:
stereo_events = stereo_series.extract_events([pars_event.duration for pars_event in parsivel_events])
all(pars_event.duration == stereo_event.duration for pars_event, stereo_event in zip(parsivel_events, stereo_events))

### Save the event for stereo 3D and parsivel

Filter for outliers

In [ ]:
outliers = [i for i, (pars_event, stereo_event) in enumerate(zip(parsivel_events, stereo_events)) if pars_event.total_depth_for_event > 10000 or stereo_event.total_depth_for_event > 10000]
for i, idx in enumerate(outliers):
    parsivel_events.pop(idx - i)
    stereo_events.pop(idx - i)
outliers

In [ ]:
pars_folder = Path("/home/debian-mmss/stage_project/data/saved_events/pars_events/")
stereo_folder = Path("/home/debian-mmss/stage_project/data/saved_events/stereo_events")
# Clear the folders from previous use
for file in pars_folder.iterdir():
    file.unlink()
for file in stereo_folder.iterdir():
    file.unlink()
for i, (pars_event, stereo_event) in enumerate(zip(parsivel_events, stereo_events)):
    pars_event.to_pickle(pars_folder / f"event{i+1:>02}.obj")
    stereo_event.to_pickle(stereo_folder / f"event{i+1:>02}.obj")